##### 2020.07.01  (ver. 2020.07.02)
##### Sathiesh's code

### fastai_02_IXI_FSL_processing

In [2]:
import os
from glob import glob
import pandas as pd
from pathlib import Path

In [3]:
root_folder = Path('/data-10tb/marek/IXI/train_data/FSL_outputs')
xls_path    = Path('/data-10tb/marek/skull-stripping-1/fastai2d/xls/IXI.xls')

In [4]:
# %load ixi_processing
### Sathiesh's code...

def process_ixi_xls(xls_path, img_path):
    print('Preprocessing ' + str(xls_path))
    df = pd.read_excel(xls_path)
    duplicate_subject_ids = df[df.duplicated(['IXI_ID'], keep=False)].IXI_ID.unique()
    for subject_id in duplicate_subject_ids:
        age = df.loc[df.IXI_ID == subject_id].AGE.nunique()
        if age != 1: df = df.loc[df.IXI_ID != subject_id]# #Remove duplicates with two different age values
    df = df.drop_duplicates(subset='IXI_ID', keep='first').reset_index(drop=True)
    df['subject_id'] = ['IXI' + str(subject_id).zfill(3) for subject_id in df.IXI_ID.values]
    df = df.rename(columns={'SEX_ID (1=m, 2=f)': 'gender', 'AGE': 'age_at_scan'})
    df = df.replace({'gender': {1:'M', 2:'F'}})
    img_list = glob(str(img_path/'*'/'T1_biascorr.nii.gz'))
    mask_list = glob(str(img_path/'*'/'T1_biascorr_brain_mask.nii.gz'))
    for path in img_list:
        subject_id = path.split('/')[-2].split('-')[0]
        df.loc[df.subject_id == subject_id, 't1_biascorr_path'] = path
    for path in mask_list:
        subject_id = path.split('/')[-2].split('-')[0]
        df.loc[df.subject_id == subject_id, 'brain_mask_path'] = path
    df = df.dropna()
    df = df[['t1_biascorr_path','brain_mask_path', 'subject_id', 'gender', 'age_at_scan']]
    df['above_60_years'] = df.age_at_scan > 60
    #os.remove(xls_path)
    return df


In [5]:
df = process_ixi_xls(xls_path, root_folder)

Preprocessing /data-10tb/marek/skull-stripping-1/fastai2d/xls/IXI.xls


In [6]:
df.head()

,t1_biascorr_path,brain_mask_path,subject_id,gender,age_at_scan,above_60_years
1,/data-10tb/marek/IXI/train_data/FSL_outputs/IX...,/data-10tb/marek/IXI/train_data/FSL_outputs/IX...,IXI002,F,35.800137,False
2,/data-10tb/marek/IXI/train_data/FSL_outputs/IX...,/data-10tb/marek/IXI/train_data/FSL_outputs/IX...,IXI012,M,38.781656,False
3,/data-10tb/marek/IXI/train_data/FSL_outputs/IX...,/data-10tb/marek/IXI/train_data/FSL_outputs/IX...,IXI013,M,46.710472,False
4,/data-10tb/marek/IXI/train_data/FSL_outputs/IX...,/data-10tb/marek/IXI/train_data/FSL_outputs/IX...,IXI014,F,34.236824,False
5,/data-10tb/marek/IXI/train_data/FSL_outputs/IX...,/data-10tb/marek/IXI/train_data/FSL_outputs/IX...,IXI015,M,24.284736,False


In [7]:
df.to_excel('xls/ixi_processed.xls', index=False)

In [8]:
pwd

'/data-10tb/marek/skull-stripping-1/fastai2d'